# 6장 Softmax를 활용한 동물 분류

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.0.0-alpha0


In [2]:
tf.random.set_seed(777) #값 고정

[![image.png](https://i.postimg.cc/FHD7q69s/image.png)](https://postimg.cc/JsHrHPD9)

* 동물의 다리 갯수 등 다양한 변수를 통해 7종류로 동물을 분류한다.
* 맨 마지막에 위치한 라벨링을 보면 0 - 6까지 7종류의 형태

In [4]:
xy = np.loadtxt('data-04-zoo.csv', delimiter = ',', dtype = np.float32)

In [9]:
xy

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 3.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 6.],
       [0., 1., 1., ..., 0., 0., 1.]], dtype=float32)

* 조작변수(X)와 라벨링(Y) 데이터를 나눠주자

In [10]:
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

* 우리의 라벨 갯수는 7개(0부터 6까지)

In [11]:
nb_classes = 7

* 라벨값이 0부터 6까지 주르륵 있다.
* 분류를 위해 라벨값을 One-Hot Encoding해서 더미처리해주자

* Tensorflow 1.X에서는 One-Hot 변환한 후, -1 처리해주는 것까지 직접했던 것 같음

[![image2.png](https://i.postimg.cc/T2JWT3xC/image2.png)](https://postimg.cc/LhhsTRn1)

In [15]:
# 반면 2.x는 필요XX
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

In [18]:
print(x_data.shape, Y_one_hot.shape)
# X는 101  X 16의 행렬, Y는 101 X 7의 행렬

(101, 16) (101, 7)


In [20]:
# 자, 우리 weight랑 bias를 셋팅
W = tf.Variable(tf.random.normal((16, nb_classes)), name ='weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')
variables = [W, b]

In [37]:
## 이제부터 Softmax 만들기 돌입

def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost


def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [38]:
def fit(X, Y, epochs = 1000, verbose = 100): #verbose는 학습 중 출력되는 문구
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        
        if (i == 0) | ((i+1) % verbose == 0):
            
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy()
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))
            
fit(x_data, Y_one_hot)

Steps: 1 Loss: 3.635028839111328, Acc: 0.1683168262243271
Steps: 100 Loss: 0.5194157958030701, Acc: 0.7920792102813721
Steps: 200 Loss: 0.31850090622901917, Acc: 0.9108911156654358
Steps: 300 Loss: 0.23534879088401794, Acc: 0.9405940771102905
Steps: 400 Loss: 0.1887214034795761, Acc: 0.9504950642585754
Steps: 500 Loss: 0.158460333943367, Acc: 0.9504950642585754
Steps: 600 Loss: 0.13703754544258118, Acc: 0.9900990128517151
Steps: 700 Loss: 0.12098979949951172, Acc: 0.9900990128517151
Steps: 800 Loss: 0.10847963392734528, Acc: 1.0
Steps: 900 Loss: 0.09843038767576218, Acc: 1.0
Steps: 1000 Loss: 0.09016558527946472, Acc: 1.0
